# Wolfram Oracle Bet example

***Before running this notebook, you might want to use Jupyter's "clear output" function to erase the results of the previous execution of this notebook. That will make more apparent what has been executed in the current session.***


[Marlowe Playground](https://play.marlowe.iohk.io//)


In Marlowe format it appears as
```
When
    [Case
        (Deposit
            (Role "Alice")
            (Role "Alice")
            (Token "" "")
            (Constant 1000)
        )
        (When
            [Case
                (Deposit
                    (Role "Bob")
                    (Role "Bob")
                    (Token "" "")
                    (Constant 1000)
                )
                (When
                    [Case
                        (Choice
                            (ChoiceId
                                "WOLF_BTCUSD"
                                (Address "addr_test1vrxx3rjangevudlrejgp9m508uc26jd02n7lmdz365n4wrca35y5j")
                            )
                            [Bound 0 100000000]
                        )
                        (If
                            (ValueLT
                                (ChoiceValue
                                    (ChoiceId
                                        "WOLF_BTCUSD"
                                        (Address "addr_test1vrxx3rjangevudlrejgp9m508uc26jd02n7lmdz365n4wrca35y5j")
                                    ))
                                (Constant 1000000)
                            )
                            (Pay
                                (Role "Alice")
                                (Party (Role "Bob"))
                                (Token "" "")
                                (Constant 1000)
                                Close 
                            )
                            (Pay
                                (Role "Bob")
                                (Party (Role "Alice"))
                                (Token "" "")
                                (Constant 1000)
                                Close 
                            )
                        )]
                    431686350279989 Close 
                )]
            216863500634957 Close 
        )]
    21686350063495 Close 
```

## Preliminaries

See [Lesson 0. Preliminaries](00-preliminaries.md) for information on setting up one's environment for using this tutorial.

The lesson assumes that the following environment variables have been set.
- `CARDANO_NODE_SOCKET_PATH`: location of Cardano node's socket.
- `CARDANO_TESTNET_MAGIC`: testnet magic number.

It also assumes that the Lender and Borrower parties have addresses, signing keys, and funds.
- Lender
    - [keys/alice.address](keys/alice.address): Cardano address for Alice
    - [keys/alice.skey](keys/alice.skey): location of signing key file for Alice
- Borrower
    - [keys/bob.address](keys/borrower.address): Cardano address for Bob
    - [keys/bob.skey](keys/borrower.skey): location of signing key file for Bob

### Access to Cardano node and Marlowe Runtime

If we're using [demeter.run](https://demeter.run/)'s Cardano Marlowe Runtime extension, then we already have access to Cardano Node and Marlowe Runtime. The followind commands will set the required environment variables to use a local docker deployment on the default ports. It will also set some supplementary environment variables.

In [9]:
if [[ -z "$CARDANO_NODE_SOCKET_PATH" ]]
then

  # Only required for `marlowe-cli` and `cardano-cli`.
  export CARDANO_NODE_SOCKET_PATH="$(docker volume inspect marlowe-starter-kit_shared | jq -r '.[0].Mountpoint')/node.socket"
  export CARDANO_TESTNET_MAGIC=2 # Note that preprod=1 and preview=2. Do not set this variable if using mainnet.

fi

# FIXME: This should have been inherited from the parent environment.
if [[ -z "$CARDANO_NODE_SOCKET_PATH" ]]
then
  export CARDANO_NODE_SOCKET_PATH=/ipc/node.socket
fi

# FIXME: This should have been set in the parent environment.
if [[ -z "$CARDANO_TESTNET_MAGIC" ]]
then
  export CARDANO_TESTNET_MAGIC=$CARDANO_NODE_MAGIC
fi

case "$CARDANO_TESTNET_MAGIC" in
  1)
    export "EXPLORER_URL=https://preprod.cardanoscan.io"
    ;;
  2)
    export "EXPLORER_URL=https://preview.cardanoscan.io"
    ;;
  *)
    # Use `mainnet` as the default.
    export "EXPLORER_URL=https://cardanoscan.io"
    ;;
esac

echo "CARDANO_NODE_SOCKET_PATH = $CARDANO_NODE_SOCKET_PATH"
echo "CARDANO_TESTNET_MAGIC = $CARDANO_TESTNET_MAGIC"

CARDANO_NODE_SOCKET_PATH = /ipc/node.socket
CARDANO_TESTNET_MAGIC = 2


In [10]:
SECOND=1000 # 1 second = 1000 milliseconds
MINUTE=$((60 * SECOND)) # 1 minute = 60 seconds
HOUR=$((60 * MINUTE)) # 1 hour = 60 minutes

NOW="$((`date -u +%s` * SECOND))"
echo NOW = "$NOW" POSIX milliseconds = "`date -d @$((NOW / SECOND))`"

ADA=1000000

NOW = 1686677526000 POSIX milliseconds = Tue Jun 13 05:32:06 PM UTC 2023


Note the test network magic number:
- `preprod` = 1
- `preview` = 2

# Run a unit test
See [test/wolfram-bet.yaml](test/wolfram-bet.yaml)

In [1]:
marlowe-cli test scripts test/wolfram-bet.yaml --faucet-key keys/faucet.skey --faucet-address $(cat keys/faucet.address) --burn-address $(cat keys/faucet.address)


AesonException "Error in $: parsing Language.Marlowe.CLI.Test.Types.ScriptTest(ScriptTest) failed, key \"stTestName\" not found"


: 1

# Examine accounts
See [Lesson 0. Preliminaries](00-preliminaries.md) to request test ADA

### Alice's address and funds

Check that an address and key has been created for the lender. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](00-preliminaries.md).

In [14]:
ALICE_SKEY=keys/alice.skey
ALICE_ADDR=$(cat keys/alice.address)
echo "ALICE_ADDR = $ALICE_ADDR"

ALICE_ADDR = addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq


Check that Alicer has at least one hundred ada.

In [16]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ALICE_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
5c5328866aa0cf7af63ffbb68b5c3a99a4cf993c56d682be8b62fcf97fcc07f9     0        75683649 lovelace + TxOutDatumNone
bd58fa8f26eeb65417080126b87a3be56dae66639dde5494bc00bbd7c49278f1     1        1000000000 lovelace + TxOutDatumNone
d6ffcd7d0dceba4236391544cd4ada8842f32878395cd088991ee2e1fe73b01c     1        1000000000 lovelace + TxOutDatumNone
d9096dd67b85d73576a5a8a3135a08dd1be1f6e26b0cbee547327b53b530b82b     0        796848386 lovelace + TxOutDatumNone
e941faa76c97cb1956d2e50ac4dc5408e0f92e355ba8e80d480ffa3a2859eb5e     0        816333124 lovelace + TxOutDatumNone


One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [17]:
echo "$EXPLORER_URL"/address/"$ALICE_ADDR"

https://preview.cardanoscan.io/address/addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq


### Bob's address and funds

Check that an address and key has been created for the borrower. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](00-preliminaries.md).

In [18]:
BOB_SKEY=keys/bob.skey
BOB_ADDR=$(cat keys/bob.address)
echo "BOB_ADDR = $BOB_ADDR"

BOB_ADDR = addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf


Check that the borrower has at least one hundred ada.

In [19]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BOB_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
37cc9570bd16281f230bd3dd1d54b43c9b6a0861692d7cc1984fe5aa9a0e642f     0        286424194 lovelace + TxOutDatumNone
77cbdf3d315ca6f6ab33adf708a511b88e269e53227f45d754d9f284833e4d84     0        78306842 lovelace + TxOutDatumNone
84b42a912f1410b7d9c5de30e145960daba47de1476760a06b27e2b4fc49e3fd     0        399777783 lovelace + TxOutDatumNone
d6ffcd7d0dceba4236391544cd4ada8842f32878395cd088991ee2e1fe73b01c     2        1000000000 lovelace + TxOutDatumNone
dcfbdc54d2f294cefddf5ddb8c81acf7c314cfba7e18759a9b49813db77b284d     0        907632079 lovelace + TxOutDatumNone


One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [20]:
echo "$EXPLORER_URL"/address/"$BOB_ADDR"

https://preview.cardanoscan.io/address/addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf


## Examine the contract

View the contract file as YAML.

In [12]:
json2yaml wolfram-contract.json

when:
- then:
    when:
    - then:
        when:
        - then:
            then:
              token:
                token_name: ''
                currency_symbol: ''
              to:
                party:
                  address: addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf
              then: close
              pay: 1000
              from_account:
                address: addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq
            if:
              value:
                value_of_choice:
                  choice_owner:
                    address: addr_test1vrxx3rjangevudlrejgp9m508uc26jd02n7lmdz365n4wrca35y5j
                  choice_name: WOLF_BTCUSD
              lt: 1000000
            else:
              token:
                token_name: ''
                currency_symbol: ''
              to:
                party:
                  address: addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq
              then: close
 

### \[Optional, but recommended\] Check the safety of the contract

If we were running the contract on the Cardano `mainnet`, then we\'d want to check its safety before creating it, so that there is no chance that we might lose funds.

Here are the steps for checking the safety of a contract:

1. Understand the [Marlowe Language](https://marlowe.iohk.io/).
2. Understand Cardano\'s [Extended UTxO Model](https://docs.cardano.org/learn/eutxo-explainer).
3. Read and understand the [Marlowe Best Practices Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/best-practices.md).
4. Read and understand the [Marlowe Security Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/security.md).
5. Use [Marlowe Playground](https://https://play.marlowe.iohk.io//) to flag warnings, perform static analysis, and simulate the contract.
6. Use [Marlowe CLI\'s](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe-cli/ReadMe.md) `marlowe-cli run analyze` tool to study whether the contract can run on a Cardano network.
7. Run *all execution paths* of the contract on a [Cardano testnet](https://docs.cardano.org/cardano-testnet/overview).

See [Lesson 1](01-runtime-cli.ipynb) for an example of performing step 6.

## Transaction 1. Create the Contract

Marlowe CLI\'s command `marlowe-cli run initialize` will build the creation information for a Marlowe contract. We provide it the JSON files containing the contract and initial state. Anyone could create the contract, but in this example the lender will be doing so, so we provide their address to fund the transaction and to receive the change from it.

In [34]:
marlowe-cli run initialize \
  --permanently-without-staking \
  --contract-file wolfram-contract.json \
  --state-file wolfram-state.json \
  --out-file marlowe-1.json

We now use Marlowe CLI\'s `marlowe-cli run auto-execute` command to construct and submit the creation transaction.

In [35]:
TX_1=$(
marlowe-cli run auto-execute \
  --marlowe-out-file marlowe-1.json \
  --change-address "$BOB_ADDR" \
  --required-signer "$BOB_SKEY" \
  --out-file tx-1.signed \
  --submit 600 \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_1 = $TX_1"


Fee: Lovelace 222217
Size: 1123 / 16384 = 6%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%
TX_1 = 6fe7cdcb4d44f5357b767df2fea740e818326a92890da34c1801f2dc878fdb69


One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [36]:
echo "$EXPLORER_URL"/transaction/"$TX_1?tab=utxo"

https://preview.cardanoscan.io/transaction/6fe7cdcb4d44f5357b767df2fea740e818326a92890da34c1801f2dc878fdb69?tab=utxo


One can also examine the contract's UTxO using `cardano-cli`.

In [24]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_1#1"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
ec39394af91ae0ea756ab729e8995ef11f27fffda8825c82eadb29f32b9b91a2     1        600000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "4e3c9e2dd5a2b3a3f813bc05254abe1ac4f3658a39f22c68ed6589ab1e5c481b"


## Transaction 2. Alice deposit

Alice is going to bet some valuable cash she was saving for a rainy day

In [37]:
marlowe-cli run prepare \
  --deposit-account "$ALICE_ADDR" \
  --deposit-party "$ALICE_ADDR" \
  --deposit-amount "100000000" \
  --invalid-before "$((`date -u +%s` * SECOND - 1 * MINUTE))" \
  --invalid-hereafter "$((`date -u +%s` * SECOND + 5 * MINUTE))" \
  --marlowe-file marlowe-1.json \
  --out-file marlowe-2.json

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1686677724000},POSIXTime {getPOSIXTime = 1686678084999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1686677724000},POSIXTime {getPOSIXTime = 1686678084999}), txInputs = [NormalInput (IDeposit "\"addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq\"" "\"addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq\"" (Token "" "") 100000000)]}


Once again, use `marlowe-cli run auto-execute` to build and submit the transaction and then wait for confirmation.

In [38]:
TX_2=$(
marlowe-cli run auto-execute \
  --tx-in-marlowe "$TX_1#1" \
  --marlowe-in-file marlowe-1.json \
  --marlowe-out-file marlowe-2.json \
  --change-address "$ALICE_ADDR" \
  --required-signer "$ALICE_SKEY" \
  --out-file tx-2.signed \
  --submit 600 \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_2 = $TX_2"


Fee: Lovelace 728055
Size: 2060 / 16384 = 12%
Execution units:
  Memory: 5990050 / 14000000 = 42%
  Steps: 1649035524 / 10000000000 = 16%
TX_2 = 63b44779d26e430be7654d3cb96007e640340c1f8d8add5400486eedc82028c9


One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [27]:
echo "$EXPLORER_URL"/transaction/"$TX_2?tab=utxo"

https://preview.cardanoscan.io/transaction/731d8ced76f3d1e97381d449b54f54418b945a3909a49b3f4fa394a0710c7755?tab=utxo


One can see that Alice has 1000 ada less than originally. 

In [39]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ALICE_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
5c5328866aa0cf7af63ffbb68b5c3a99a4cf993c56d682be8b62fcf97fcc07f9     0        75683649 lovelace + TxOutDatumNone
63b44779d26e430be7654d3cb96007e640340c1f8d8add5400486eedc82028c9     0        798544566 lovelace + TxOutDatumNone
d6ffcd7d0dceba4236391544cd4ada8842f32878395cd088991ee2e1fe73b01c     1        1000000000 lovelace + TxOutDatumNone
d9096dd67b85d73576a5a8a3135a08dd1be1f6e26b0cbee547327b53b530b82b     0        796848386 lovelace + TxOutDatumNone
e941faa76c97cb1956d2e50ac4dc5408e0f92e355ba8e80d480ffa3a2859eb5e     0        816333124 lovelace + TxOutDatumNone


In [40]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_2#1"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
63b44779d26e430be7654d3cb96007e640340c1f8d8add5400486eedc82028c9     1        700000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "072cbca1b638831001626ab64306bc09df49d7b6366614dde6a5cc6002c3346f"


## Transaction 3. Bob deposit

Bob is unemployed because of his gampling problems. This bet is his only chance to find a good therapist.

In [41]:
marlowe-cli run prepare \
  --deposit-account "$BOB_ADDR" \
  --deposit-party "$BOB_ADDR" \
  --deposit-amount "100000000" \
  --invalid-before "$((`date -u +%s` * SECOND - 1 * MINUTE))" \
  --invalid-hereafter "$((`date -u +%s` * SECOND + 5 * MINUTE))" \
  --marlowe-file marlowe-2.json \
  --out-file marlowe-3.json

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1686677803000},POSIXTime {getPOSIXTime = 1686678163999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1686677803000},POSIXTime {getPOSIXTime = 1686678163999}), txInputs = [NormalInput (IDeposit "\"addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf\"" "\"addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf\"" (Token "" "") 100000000)]}


Once again, use `marlowe-cli run auto-execute` to build and submit the transaction and then wait for confirmation.

In [42]:
TX_3=$(
marlowe-cli run auto-execute \
  --tx-in-marlowe "$TX_2#1" \
  --marlowe-in-file marlowe-2.json \
  --marlowe-out-file marlowe-3.json \
  --change-address "$BOB_ADDR" \
  --required-signer "$BOB_SKEY" \
  --out-file tx-3.signed \
  --submit 600 \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_3 = $TX_3"


Fee: Lovelace 692482
Size: 1840 / 16384 = 11%
Execution units:
  Memory: 5694984 / 14000000 = 40%
  Steps: 1554116205 / 10000000000 = 15%
TX_3 = 7acbf1f09c29bbd0c2be9a66924be1284f58e694973c9914e8788f96ecbce0ed


One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [43]:
echo "$EXPLORER_URL"/transaction/"$TX_3?tab=utxo"

https://preview.cardanoscan.io/transaction/7acbf1f09c29bbd0c2be9a66924be1284f58e694973c9914e8788f96ecbce0ed?tab=utxo


# Now we wait for Oracle to decide the winner

In [46]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ALICE_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
5c5328866aa0cf7af63ffbb68b5c3a99a4cf993c56d682be8b62fcf97fcc07f9     0        75683649 lovelace + TxOutDatumNone
63b44779d26e430be7654d3cb96007e640340c1f8d8add5400486eedc82028c9     0        798544566 lovelace + TxOutDatumNone
d6ffcd7d0dceba4236391544cd4ada8842f32878395cd088991ee2e1fe73b01c     1        1000000000 lovelace + TxOutDatumNone
d9096dd67b85d73576a5a8a3135a08dd1be1f6e26b0cbee547327b53b530b82b     0        796848386 lovelace + TxOutDatumNone
e941faa76c97cb1956d2e50ac4dc5408e0f92e355ba8e80d480ffa3a2859eb5e     0        816333124 lovelace + TxOutDatumNone


The borrower now has about 5 ada (the loan's interest) less than originally.

In [47]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BOB_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
6fe7cdcb4d44f5357b767df2fea740e818326a92890da34c1801f2dc878fdb69     0        307409862 lovelace + TxOutDatumNone
77cbdf3d315ca6f6ab33adf708a511b88e269e53227f45d754d9f284833e4d84     0        78306842 lovelace + TxOutDatumNone
7acbf1f09c29bbd0c2be9a66924be1284f58e694973c9914e8788f96ecbce0ed     0        185731712 lovelace + TxOutDatumNone
84b42a912f1410b7d9c5de30e145960daba47de1476760a06b27e2b4fc49e3fd     0        399777783 lovelace + TxOutDatumNone
ec39394af91ae0ea756ab729e8995ef11f27fffda8825c82eadb29f32b9b91a2     0        399777783 lovelace + TxOutDatumNone


# Run Mini-Oracle manually

In [60]:
TICKER=$(curl "https://api.wolframalpha.com/v1/result?appid=6WU6JX-46EP5U9AGX&i=1%20btc%20to%20usd%20number" | sed 's/[^0-9]//g')
echo "TICKER = $TICKER"

marlowe-cli run prepare \
  --choice-name "WOLF_BTCUSD" \
  --choice-party "addr_test1vrxx3rjangevudlrejgp9m508uc26jd02n7lmdz365n4wrca35y5j" \
  --choice-number "$TICKER" \
  --invalid-before "$((`date -u +%s` * SECOND - 1 * MINUTE))" \
  --invalid-hereafter "$((`date -u +%s` * SECOND + 5 * MINUTE))" \
  --marlowe-file marlowe-3.json \
  --out-file marlowe-4.json


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    22    0    22    0     0     25      0 --:--:-- --:--:-- --:--:--    25
TICKER = 25900
Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1686681824000},POSIXTime {getPOSIXTime = 1686682184999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1686681824000},POSIXTime {getPOSIXTime = 1686682184999}), txInputs = [NormalInput (IChoice (ChoiceId "WOLF_BTCUSD" "\"addr_test1vrxx3rjangevudlrejgp9m508uc26jd02n7lmdz365n4wrca35y5j\"") 25900)]}
Payment 1
  Acccount: "\"addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq\""
  Payee: Party "\"addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf\""
  Ada: Lovelace {getLovelace = 100000000}
Payment 2
  Acccount: "\"addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf\""
  Payee: Party "\"addr_test1vrudhk63pclytll6vdt

In [61]:
TX_4=$(
marlowe-cli run auto-execute \
  --tx-in-marlowe "$TX_3#1" \
  --marlowe-in-file marlowe-3.json \
  --marlowe-out-file marlowe-4.json \
  --change-address "addr_test1vrxx3rjangevudlrejgp9m508uc26jd02n7lmdz365n4wrca35y5j" \
  --required-signer "keys/oracle.skey" \
  --out-file tx-4.signed \
  --submit 600 \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_4 = $TX_4"


Fee: Lovelace 668923
Size: 1090 / 16384 = 6%
Execution units:
  Memory: 5861728 / 14000000 = 41%
  Steps: 1554062751 / 10000000000 = 15%
TX_4 = 5de014d153d186aec12a0bc271d8daea052846c28756d662e36c52430575adde


In [62]:
echo "$EXPLORER_URL"/transaction/"$TX_3?tab=utxo"

https://preview.cardanoscan.io/transaction/7acbf1f09c29bbd0c2be9a66924be1284f58e694973c9914e8788f96ecbce0ed?tab=utxo
